In [2]:
import pandas as pd
import numpy as np

print('instance started!')

instance started!


In [9]:
df = pd.read_csv('housing.csv')
#df
col = ['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value']
sub_df = df.loc[(df.ocean_proximity == '<1H OCEAN') | (df.ocean_proximity == 'INLAND'), col]

In [13]:
sub_df.isna().sum()

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64